# ESE-2000 Lab 6
TO DO add intro

We download a ~1MB file containing the entirety of Shakespeare's work. This is the dataset we will train our language model on.

In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
from IPython.core.display_functions import clear_output
import matplotlib.pyplot as plt
import math
import wandb
import re
from tqdm import tqdm
device = "cpu"
# if torch.backends.mps.is_available():
#     device = "mps:0"
# elif torch.cuda.is_available():
#     device = "cuda:0" 
print(f"Using device: {device}")

Using device: cpu


In [53]:
# # deleteme
# dd=torch.ones(5,5)
# mask = torch.tril_indices(5,5, -1)
# dd[mask[0], mask[1]] = float('-inf')
# dd

### Data

###  Data Loading

In [54]:
with open('input.txt') as f:
    text = f.read()
    

print("----Sample Shakespeare----")
print(text[:250])

----Sample Shakespeare----
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



### Tokenization

Tokenization converts raw sub-sequences of text (substrings) to sequences of integers. For example, `"ll." -> 208`. We will be developing a character level language model, so we will be converting each individual word into an integer. For example, `"Hello" -> 48`.

In [55]:
def split_to_words(text):
    return re.findall(r"\w+(?:'\w+)?|[.,!?;:\"()\[\]{}<>\\/\-—–…]|\n", text)

vocab = list(set(split_to_words(text)))
c = len(vocab)
print("Number of words: {}".format(len(split_to_words(text))))
print("Number of distinct words in text: {}".format(c))

Number of words: 292072
Number of distinct words in text: 14295


### Functions to encode and decode words into indices

In [56]:
stoi = {word:i for i, word in enumerate(vocab)}
itos = {i:word for i, word in enumerate(vocab)}

def words_to_tokens(words):
    """
    Convert a list of words to a list of tokens
    """
    return [stoi[w] for w in words]

def tokens_to_words(index_list):
    """
    Convert a list of tokens to a list of words
    """
    decoded = " ".join([itos[i] for i in index_list])
    return re.sub(r'\s+([.,!?;:"(){}\[\]<>\\/\-—–…])', r'\1', decoded)

# Checking that the word to token and back conversion works
sample_words = text[:36]
token_ids = words_to_tokens(split_to_words(sample_words))
recovered_words = tokens_to_words(token_ids)
print(f"Original text: {sample_words}\n")
print(f"Encoded text: {token_ids}\n")
print(f"Recovered text: {recovered_words}\n")

Original text: First Citizen:
Before we proceed any

Encoded text: [11164, 13785, 101, 178, 3687, 1787, 6423, 844]

Recovered text: First Citizen: 
 Before we proceed any



### Converting dataset into tokens

In [57]:
tokenized_text = words_to_tokens(split_to_words(text))
print("Encoded text sample: {}".format(tokenized_text[:10]))
print(tokens_to_words(tokenized_text[:10]))

# The works of Shakespeare are now a sequence of integers representing the words in the text. Sorry, William.
tokenized_text = torch.tensor(tokenized_text)
tokenized_text.shape

Encoded text sample: [11164, 13785, 101, 178, 3687, 1787, 6423, 844, 9168, 6637]
First Citizen: 
 Before we proceed any further,


torch.Size([292072])

## Task 2: Co-occurrence matrix

In [58]:
#TODO commented bc its slow
# Create co-occurrence matrix
# The co-occurrence matrix C is a c x c (c is our vocab size) symmetric matrix where C_ij is how many times the ith word appears within W words away from the jth word.
# W = 10
# C = torch.stack([torch.zeros(len(vocab)) for _ in range(len(vocab))])
# for t_idx in trange(len(tokenized_text)):
#     left_bound = max(t_idx-W//2,0)
#     right_bound = min(t_idx+W//2+1,len(tokenized_text))
#     context_words = tokenized_text[left_bound : right_bound]
#     for u_idx in range(left_bound, right_bound):
#         t = tokenized_text[t_idx]
#         u = tokenized_text[u_idx]
#         C[t, u] += 1.0
# C = C.to(device)
# # X should be a symmetric matrix
# torch.isclose(C, C.T, atol=1e-3).all()

# # Save C so that we dont have to compute it again
#torch.save(C, "C.pt")

# Load C from storage
C = torch.load("C.pt").to(device)

/tmp/ipykernel_199231/504936921.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  C = torch.load("C.pt").to(device)


In [59]:
# size of C in GB: numel times 4 bytes per float / 1e9 which is GB
C.numel() * 4 / 1e9

0.8173881

In [60]:
torch.isclose(C, C.T, atol=1e-3).all()

tensor(True)

## Task 3: PCA Embeddings

In [82]:
# Torch has a bug on mps devices so this won't work on MacBooks
n = 256
# with torch.no_grad():
#     Z = C - C.mean(dim=1, keepdim=True)
#     Z /= Z.std(dim=1, keepdim=True)
#     cov = (Z @ Z.T)/(Z.shape[0] - 1)
#     L, Q = torch.linalg.eigh(cov)
#     principal_eigv = Q[:, -n:].T

#     # PCA embeddings for training
#     pca_embeddings = Z @ principal_eigv.T # (c, n)
#     # Full pca_embeddings if we need them to visualize
#     # In vector form would be Q.T @ x_n
#     full_embeddings = Z @ Q

# torch.save(embeddings, "embeddings.pt")
# Load embeddings
pca_embeddings = torch.load("embeddings.pt").to(device)
pca_embeddings.shape

/tmp/ipykernel_199231/3551890280.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pca_embeddings = torch.load("embeddings.pt").to(device)


torch.Size([14295, 256])

## Visualize embeddings
Decide if this section goes into the notebook or not

### Average coefficients


In [30]:
# K=8192
# #average_coefficients = full_embeddings.mean(axis=0)
# sample_embeddings = full_embeddings[torch.randint(0,full_embeddings.shape[0],(1000,))]
# # Compute the expectation of the absolute value of the norm of each component.
# average_coefficients = sample_embeddings.norm(p=2,dim=0).cpu().numpy()[::-1]
# data = average_coefficients[:K]

# # Reverse the tensor:
# data = data

# # Normalize by sum?
# #data = data / data.sum()

# plt.figure(figsize=(12, 6))
# plt.title(f"Average Coefficients (k={K})")
# fig= plt.plot(range(K), data,marker='.',linestyle='')


# plt.show()

### Principal eigenvalues

In [31]:
# K=64
# L_plot = L[-K:]/L.sum()
# L_plot,_ = L_plot.sort(descending=True)
# L_plot = L_plot.cpu().numpy()
# plt.figure(figsize=(12, 6))
# plt.title("Top k eigenvalues (k=64)")
# markerline, stemlines, baseline = plt.stem(range(K), L_plot, linefmt='b-', markerfmt='o', basefmt='k-')
# plt.setp(markerline, marker='o', fillstyle='none')


### Co ocurrence matrix plot

In [32]:
# # Top 10 words
# top_10_words = C.sum(axis=0).sort(descending=True).indices[:10]
# top_10_words = [vocab[i] for i in top_10_words]
# print(top_10_words)

In [33]:
# import matplotlib.pyplot as plt
# import numpy as np

# # # Remove all the punctations and stop words from the matrix for visualization
# X_viz = C.clone()
# words_to_remove = [",", ":", ".", "the", "I", "to", "and", ";", "of", "you", "my", "a", "?", "!", "in", "that", "And", "not", "-", "is", "me", "be", "it", "with", "your", "for", "his", "have", "this", "thou", "as", "thy", "will", "so", "but", "The", "To", "all", "her", "thee", "by", "are", "our", "do", "we"]
# vocab_to_remove_indices = set(words_to_tokens(words_to_remove))
# idx_to_viz = [i for i, word in enumerate(vocab) if word not in vocab_to_remove_indices]
# X_viz = X_viz[idx_to_viz, :][:, idx_to_viz]

# # top 20 words not including stop words
# top_100_words = C.sum(axis=0).sort(descending=True).indices[:100].cpu().numpy()
# top_100_nostop = [word for word in top_100_words if word not in vocab_to_remove_indices]
# display(f"Top 100 words, excluding punctation and most common stop words: {tokens_to_words(top_100_nostop)}")

# # Create a custom colormap
# cmap = plt.cm.get_cmap('viridis').copy()
# cmap.set_over('green')

# # Plot the image with the custom colormap
# plt.figure(figsize=(10, 10))
# plt.imshow(X_viz.cpu().numpy(), cmap=cmap, vmax=3)

# # Add colorbar with custom settings
# cbar = plt.colorbar(extend='max')
# cbar.set_label('Value')

# plt.title('Co-occurrence Matrix')
# plt.show()
# # # Model

In [34]:
# test_loss = 0
# with torch.no_grad():
#     for t_idx, (E, y) in enumerate(test_loader):
#         logits = model(E)
#         B, _ = logits.shape
#         loss = F.cross_entropy(logits, y)
#         test_loss += loss.item()

# print("Test loss: ", test_loss / len(test_loader))

In [35]:
# initial = test[132:132+T].unsqueeze(0)
# generated_text = generate(model,initial, max_generate_tokens=100)
# print("\n===INPUT===\n")
# print(tokens_to_words(initial.reshape(-1).tolist()))
# print("\n===GENERATED TEXT===\n")
# print("".join(generated_text[:]))

## Task 4: Language Transformer

### MultiHeadLayer

In [62]:
# This is the same as the MultiheadLayer in the lab 6 notebook. It corresponds to the equations in Section 3 of this lab's writeup.
class MultiHeadLayer(nn.Module):
    """
    An implementation of the multihead attention layer.
    The difference between AttentionLayer and this class is,
    now Q,K,V are matrices of shape (H, m, n), and the attention matrix B is of shape (H, T, T)
    (one attention feature per head)
    Args:
        m (int): The dimension of the Q and K matrices.
        n (int): The number of features, n=12 in our case.
        k (int): The dimension of the W matrix.
        H (int): The number of heads.
    """
    def __init__(self, m, n, H):
        super(MultiHeadLayer, self).__init__()
        self.m = m
        self.H = H

        self.Q = nn.Parameter(torch.empty(H, m, n))
        self.K = nn.Parameter(torch.empty(H, m, n))
        self.V = nn.Parameter(torch.empty(H, m, n))

        self.W = nn.Parameter(torch.empty(H, n, m))
        
        self.nonlinearity = nn.ReLU()
        self.initialize_parameters()

    def initialize_parameters(self):
        """
        Initialize the values of the learnable parameter matrices.
        Kaiming uniform is just a type of random initialization, you don't need to 
        worry about it. It is a good default initialization for linear layers.
        """
        nn.init.kaiming_uniform_(self.Q, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.K, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.V, a=math.sqrt(5))

        nn.init.kaiming_uniform_(self.W, a=math.sqrt(5))

    def forward(self, X):
        """
        Args:
            X (torch.Tensor): The input embeddings.
        Returns:
            X_l (torch.Tensor): The output of the multihead attention layer.
        """
        B, n, T = X.shape  # X: (B, n, T)

        # Expand X to include the head dimension
        X_expanded = X.unsqueeze(1)  # (B, 1, n, T)

        # Compute QX, KX, VX for each head
        QX = torch.matmul(self.Q.unsqueeze(0), X_expanded)  # (B, H, m, T)
        KX = torch.matmul(self.K.unsqueeze(0), X_expanded)  # (B, H, m, T)
        VX = torch.matmul(self.V.unsqueeze(0), X_expanded)  # (B, H, m, T)
        
        QX_t = QX.transpose(-2, -1)  # (B, H, T, m)

        # Compute attention scores B per head
        B = torch.matmul(QX_t, KX)  # (B, H, T, T)
        A = F.softmax(B, dim=-1)
    
        A_t = A.transpose(-2,-1)
        VXA_t = torch.matmul(VX, A_t) # (B, H, m, T)
        Y = torch.matmul(self.W, VXA_t) # (B, H, T, n)
        
        X_l = X + self.nonlinearity(Y.sum(dim=1))

        return X_l
    
    
model = MultiHeadLayer(m=32, n=256, H=8).to(device)
X_tilde = torch.randn(1,256,64).to(device)
out = model(X_tilde)

print(f"out.shape: {out.shape}")

out.shape: torch.Size([1, 256, 64])


### LanguageTransformer

In [63]:
class LanguageTransformer(nn.Module):
    """
    
    Mutlihead Transformer, analogous to the Transformer class, in the single head case.
    Args:
        m (int): The dimension of the Q and K matrices.
        n (int): The number of features, n=12 in our case.
        k (int): The dimension of the W matrix.
        L (int): The number of layers.
        H (int): The number of heads.
    """
    def __init__(self, m, n, L, H):
        super(LanguageTransformer, self).__init__()
        self.layers = nn.ModuleList([
            MultiHeadLayer(m, n, H) for _ in range(L)
        ])
        # Word embedding table. This is the only change from the previous lab's code. We have 
        # PCA embeddings to convert word indices to embeddings.
        self.embedding_table = pca_embeddings
        
    def forward(self, E):
        """
        The forward pass of the multihead transformer, stacks L multihead layers.
        This class is essentially the same as the Transformer class, but using the 
        MultiHeadLayer class instead of the AttentionLayer class.
        Args:
            E (torch.Tensor): The input word indices.
        Returns:
            X_L^{T-1} (torch.Tensor): The last vector of the output of the transformer.
        """
        # Convert word indices to embeddings. We need to transpose the result to get the shape (B, n, T).
        X = self.embedding_table[E].transpose(1,2)
        B, n, T = X.shape

        # Compute the mean token to append to the sequence.
        X_tilde = X.mean(dim=2, keepdim=True) # mean over the time dimension
        X_tilde = torch.cat((X, X_tilde), dim=-1)
        
        # X_l has shape (B, n, T+1)
        X_l = X_tilde
        for layer in self.layers:
            X_l = layer(X_l)
        
        # Output the last vector.
        return X_l[:,:,-1]

# Test
model = LanguageTransformer(L=2, H=2, m=32, n=256).to(device)
E = torch.randint(0, pca_embeddings.shape[0], (1,5)).to(device).long()
out = model(E)
print(f"output.shape: {out.shape}")

output.shape: torch.Size([1, 256])


### Data Split

In [64]:
T = 64 # context size
split_factor = 0.9
split_index = int(split_factor * len(tokenized_text))
    
# Splitting into train and test sets
train = tokenized_text[:split_index].to(device)
test = tokenized_text[split_index:].to(device)

### Dataset

In [65]:
class WordIndexDataset(Dataset):
    """
    This Dataset class takes and encoded tensor of word indices and returns a tensor of context windows of size T.
    The tensors returned by this dataset are not yet one-hot encoded.
    """
    def __init__(self, text, T):
        self.text = text
        self.T = T
        assert self.T < len(text), "context_size (T) must be less than len(text)"

    def __len__(self):
        return len(self.text) - self.T

    def __getitem__(self, idx):
        """
        Return a single context window of size T. 
        The context window is a sequence of T words.

        During training, we will predict the next token of every word in the context window,
        so Y_item is the next word for every word in the context window.
        """
        X_item = self.text[idx:idx + self.T]
        Y_item = self.text[idx + 1:idx + self.T + 1]

        return X_item, Y_item

train_dataset = WordIndexDataset(train, T)
test_dataset = WordIndexDataset(test, T)


# Example of a batch
B = 64
train_loader = DataLoader(train_dataset, batch_size=B, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=B, shuffle=False)

E, y_idx = next(iter(train_loader))
print(f"X_idx shape: {E.shape}")
print(f"y_idx shape: {y_idx.shape}")

X_idx shape: torch.Size([64, 64])
y_idx shape: torch.Size([64, 64])


## Task 5: Training on word embeddings

In [66]:
# Training
n_epochs = 10
m = 32
n = 256
L = 6
T = T
H = 8

estimator = LanguageTransformer(m, n, L, H).float().to(device)
optimizer = torch.optim.SGD(estimator.parameters(), lr=1e-5)

cross_entropy_loss = nn.MSELoss()
estimator.train()
train_loss = []

for epoch in range(n_epochs): # Iterate over n_epochs epochs

    for x_batch, y_batch in tqdm(train_loader): # Iterate over all batches in the dataset 
        # Load the embeddings for the target word
        # We want to predict the last word of the context window for this exercise.
        y_word_to_predict = y_batch[:,-1]
        Y_embeddings = pca_embeddings[y_word_to_predict].transpose(0,1).to(device) # (B, n)
        
        # (Step i) Load the data. These commands send the data to the GPU memory.
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        # (Step ii) Compute the gradients. We use automated differentiation.
        optimizer.zero_grad() # Gradient reset to indicate where the backward computation stops.

        # Call the neural network. In this case, we will take the average of the output of the
        # transformer as the prediction.
        y_hat = estimator(x_batch).mean(dim=-1)
        cross_entropy_value = cross_entropy_loss(y_hat,Y_embeddings)

        cross_entropy_value.backward() # Compute gradients moving backwards untit the gradient reset.

        # (Step iii) Update parameters by taking an SGD (or other optimizer) step.
        optimizer.step()

        train_loss.append(cross_entropy_value.item())
    if epoch % 5 == 0:
        print(f"Epoch {epoch}/{n_epochs} Loss: {train_loss[-1]}")

    # End of batch loop.

print(train_loss[-1]) # Print training loss.

  0%|          | 0/4107 [00:00<?, ?it/s]

/home/jporras/miniconda3/envs/lab_5a_trf/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([256, 64])) that is different to the input size (torch.Size([64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  0%|          | 13/4107 [00:03<19:07,  3.57it/s]


KeyboardInterrupt: 

## Task 6: Adding the readout layer

In [67]:
class LanguageTransformerWithReadout(nn.Module):
    """
    A slight modification of the LanguageTransformer class of Task 4.
    Args:
        m (int): The dimension of the Q and K matrices.
        n (int): The number of features, n=12 in our case.
        k (int): The dimension of the W matrix.
        L (int): The number of layers.
        H (int): The number of heads.
        c (int): The vocabulary size.
    """
    def __init__(self, m, n, L, H, c):
        super(LanguageTransformerWithReadout, self).__init__()
        self.layers = nn.ModuleList([
            MultiHeadLayer(m, n, H) for _ in range(L)
        ])

        self.embedding_table = pca_embeddings

        # Adding readout layer
        self.readout = nn.Parameter(torch.empty(c, n).to(device))
        nn.init.kaiming_uniform_(self.readout, a=math.sqrt(5))
        
    def forward(self, E):
        """
        We change the forward pass from the previous Transformer.
        Instead of concatenating a vector to the sequence, we now output a vector of probabilities for each word in the sequence.
        Args:
            E (torch.Tensor): The input word indices.
        Returns:
            Y_hat (torch.Tensor): The output of the transformer, passed through the readout layer.
        """
        X = self.embedding_table[E].transpose(1,2)

        B, n, T = X.shape
        
        # X_l has shape (B, n, T+1)
        X_l = X
        for layer in self.layers:
            X_l = layer(X_l)

        # We implement the readout layer as a linear mapping on each word in the sequence.
        Y_hat = torch.matmul(self.readout, X_l) # (B, c, T)

        # Notice, we don't apply the softmax here, because we keep the probabilities unnormalized until 
        # we call the loss function, for numerical stability.
        return Y_hat

# testing. Now the transformer outputs a vector of probabilities for each word in the sequence.
E = torch.randint(0, len(vocab), (1,5)).to(device).long()
model = LanguageTransformerWithReadout(m=32, n=256, L=6, H=8, c=c).to(device)
out = model(E)
print(f"out.shape: {out.shape}")

out.shape: torch.Size([1, 14295, 5])


## Task 7: Training for word probabilities

In [68]:
# Training
n_epochs = 5
m = 32
n = 256
L = 6
T = T
H = 8

estimator = LanguageTransformerWithReadout(m, n, L, H, c).float().to(device)
optimizer = torch.optim.SGD(estimator.parameters(), lr=1e-5)

# We use the Cross Entropy loss for estimating the probabilities of the next word.
cross_entropy_loss = nn.CrossEntropyLoss()
estimator.train()
train_loss = []

for epoch in range(n_epochs): # Iterate over n_epochs epochs

    for x_batch, y_batch in tqdm(train_loader): # Iterate over all batches in the dataset 
        # We want to predict tha last word of the context window for this exercise.
        y_word_to_predict = y_batch[:,-1]
        # Load the embeddings for the words.
        X_embeddings = pca_embeddings[x_batch].transpose(1,2) # (B, n, T)
        
        # (Step i) Load the data. These commands send the data to the GPU memory.
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        # (Step ii) Compute the gradients. We use automated differentiation.
        optimizer.zero_grad() # Gradient reset to indicate where the backward computation stops.

        # Call the neural network. In this case, we will take the average of the output of the
        # transformer as the prediction.
        y_hat = estimator(x_batch)

        # When using cross entropy loss, we need to pass the target as a 1D tensor of class indices.
        # The softmax function is applied internally to the transformer's output y_hat.
        cross_entropy_value = cross_entropy_loss(y_hat,y_batch)

        cross_entropy_value.backward() # Compute gradients moving backwards untit the gradient reset.

        # (Step iii) Update parameters by taking an SGD (or other optimizer) step.
        optimizer.step()

        train_loss.append(cross_entropy_value.item())
    if epoch % 5 == 0:
        print(f"Epoch {epoch}/{n_epochs} Loss: {train_loss[-1]}")

    # End of batch loop.

print(train_loss[-1]) # Print training loss.

  0%|          | 0/4107 [00:00<?, ?it/s]

  0%|          | 11/4107 [00:07<48:01,  1.42it/s]


KeyboardInterrupt: 

## Task 8: Sampling from a language model

In [69]:
# Taking a snippet of the text set to test the model.
starting_point = torch.randint(0, len(test)-T, (1,))
initial_indices = test[starting_point:starting_point+T].unsqueeze(0)

log_probabilities = model(initial_indices)
print(f"log_probabilities.shape: {log_probabilities.shape}")
last_word_probabilities = log_probabilities[:,:,-1]
probabilities = F.softmax(last_word_probabilities, dim=-2)

print(f"Input text: {tokens_to_words(initial_indices.reshape(-1).tolist())}")
print(f"\nThe most likely next word is: {tokens_to_words([torch.argmax(probabilities).item()])}")


print("\nSampled words according to a multinomial distribution (either could be the next word when using sampling):")
for _ in range(10):
    sampled_word = torch.multinomial(probabilities, num_samples=1).item()
    print(f"{tokens_to_words([sampled_word])}", end=" ")


log_probabilities.shape: torch.Size([1, 14295, 64])
Input text: afeard of you. 
 
 Widow: 
 He that is giddy thinks the world turns round. 
 
 PETRUCHIO: 
 Roundly replied. 
 
 KATHARINA: 
 Mistress, how mean you that? 
 
 Widow: 
 Thus I conceive by him. 
 
 PETRUCHIO: 
 Conceives by me! How likes Hortensio that?

The most likely next word is: Blunt

Sampled words according to a multinomial distribution (either could be the next word when using sampling):
warning cuff vilely foulest building keep'st starveth chose assist nails 

## Task 9: Generating language

In [70]:
def generate_text(model, X, max_generate_tokens=500):
    """
    Generate text from a model given an initial input token sequence.
    Args:
        model (nn.Module): The model to use for generation.
        input_tokens (torch.Tensor): The initial input token sequence.
        max_generate_tokens (int): The maximum number of tokens to generate.
    Returns:
        torch.Tensor: The generated token sequence.
    """
    with torch.no_grad():
        context = X.clone()
        generated_sequence = X.cpu().squeeze().tolist()  # Ensure it's a 1D list
        for _ in range(max_generate_tokens):
            logits = model(context)
            
            last_word_embeddings = logits[:,:,-1]
            probs = F.softmax(last_word_embeddings, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            
            # Slide context window: remove the first token and append the next token
            context = torch.cat([context[:, 1:], next_token], dim=1)  
            generated_sequence.append(next_token.squeeze().item())  # Add new token to generated sequence
        generated_words = tokens_to_words(generated_sequence)
        generation_string = "".join(generated_words)
        return generation_string

# Test generate
#model = LanguageTransformerWithReadout(m=32, n=256, L=6, H=8, c=c).to(device)
starting_point = torch.randint(0, len(test)-T, (1,))
initial_indices = test[starting_point:starting_point+T].unsqueeze(0)
print(f"========== INPUT TEXT ==========")
print(f"{tokens_to_words(initial_indices.reshape(-1).tolist())}\n")

# This is the model from task 7
print(f"========== INPUT + GENERATED TEXT ==========")
print(generate_text(model, initial_indices, max_generate_tokens=100))
print(f"====================================")


========== INPUT TEXT ==========
thou shalt think on prating whilst thou livest! 
 I tell thee, I, that thou hast marr'd her gown. 
 
 Tailor: 
 Your worship is deceived; the gown is made 
 Just as my master had direction: 
 Grumio gave order how it should be done. 
 
 GRUMIO: 
 I gave him no

========== INPUT + GENERATED TEXT ==========
thou shalt think on prating whilst thou livest! 
 I tell thee, I, that thou hast marr'd her gown. 
 
 Tailor: 
 Your worship is deceived; the gown is made 
 Just as my master had direction: 
 Grumio gave order how it should be done. 
 
 GRUMIO: 
 I gave him no endow'd enter'd cross obscure approved minds sol buckets Wherewith paste hose ass readiest tormentors rise whistle cheated Stephen dependant suppliants Wrath assembled prisoner's attendance cries wreaths burr sternness lilies Mantua's cur stood gotten cracking valiantness rings thankless derivative holiness incorporate policy accomplish'd beaten retail Twould dogg'd gallery Whipp'd amity steely r

In [71]:
words = "To be or not to be".split(" ")
initial_indices = torch.tensor(words_to_tokens(words)).to(device).unsqueeze(0)
print(generate_text(model, initial_indices, max_generate_tokens=100))

To be or not to be did Resign repent officers chestnut ladyship cross weeping desperation words steel'd murder'd rebukes worthiness possess'd unloads chair crushing increaseth generation comely lying shivering memory vesture stair graft bow assembled slaughter'd hospitable wooer's You're bluntly flaky piled annual conferring frank esteem carriage began Thracian esteem tending effect glove childish scoffs temper'd unfit Call'd destroyed force nymphs Likely helpless props painter length wield revenge inducement vicar mortality needed pamper horizon frequent Hercules forfend Had dad stinted Post load Humbly spotted circled Wars jay racking Tybalt frosts shoot beguile looking sorrow shoe Nathaniel solemn Turkey Twould whether Conceiving Pudding nonage presage Served wombs


## Task 10: Positional encoding

In [72]:
class LanguageTransformerWithReadoutAndPositionalEncoding(nn.Module):
    """
    Modification of the LanguageTransformerWithReadout class of Task 7 to include positional encoding.
    Positional encoding is a learnable matrix that is added to the embeddings of the input tokens.
    
    Each entry in the positional encoding matrix is is a vector of size n that represents a position in the sequence.
    """
    def __init__(self, m, n, L, H, c):
        super(LanguageTransformerWithReadoutAndPositionalEncoding, self).__init__()
        self.layers = nn.ModuleList([
            MultiHeadLayer(m, n, H) for _ in range(L)
        ])

        # Learnable parameters for positional encoding. 
        # Each entry in the positional encoding matrix is is a vector of size n that represents a position in the sequence.
        self.position_embedding = nn.Embedding(T, n)

        self.embedding_table = pca_embeddings

        # Adding readout layer
        self.readout = nn.Parameter(torch.empty(c, n).to(device))
        nn.init.kaiming_uniform_(self.readout, a=math.sqrt(5))
        
    def forward(self, E):
        """
        We change the forward pass from the previous Transformer.
        Instead of concatenating a vector to the sequence, we now output a vector of probabilities for each word in the sequence.
        Args:
            E (torch.Tensor): The input word indices.
        Returns:
            Y_hat (torch.Tensor): The output of the transformer, passed through the readout layer.
        """
        B, T = E.shape

        # Word embeddings
        X = self.embedding_table[E].transpose(1,2) # (B, n, T)

        # To create positional encodings, we need to create a vector for each position in the sequence.
        P = self.position_embedding(torch.arange(T, device=device)).transpose(0,1) # (n, T)
        
        # Adding word embeddings and positional encoding
        # Although P is (n,T), this is broadcasted to (B, n, T), which means that the same 
        # positional encoding is added to every sequence in the batch.
        X_tilde = X + P
        
        # X_l has shape (B, n, T+1)
        X_l = X_tilde
        for layer in self.layers:
            X_l = layer(X_l)

        # We implement the readout layer as a linear mapping on each word in the sequence.
        Y_hat = torch.matmul(self.readout, X_l) # (B, c, T)

        # Notice, we don't apply the softmax here, because we keep the probabilities unnormalized until 
        # we call the loss function, for numerical stability.
        return Y_hat

# testing. Now the transformer outputs a vector of probabilities for each word in the sequence.
E = torch.randint(0, len(vocab), (1,5)).to(device).long()
print(f"E.shape: {E.shape}")
model = LanguageTransformerWithReadoutAndPositionalEncoding(m=32, n=256, L=6, H=8, c=c).to(device)
out = model(E)
print(f"out.shape: {out.shape}")

E.shape: torch.Size([1, 5])
out.shape: torch.Size([1, 14295, 5])


## Task 11: Layer Normalization

In [89]:
# We now need to modify both MultiHeadLayer and the LanguageTransformer class to include layer normalization.
class MultiHeadLayer(nn.Module):
    """
    A modified version of the MultiHeadLayer class with layer normalization.
    It will have two normalization layers, one after the multi-head attention and one after the nonlinearity.
    """
    def __init__(self, m, n, H):
        super(MultiHeadLayer, self).__init__()
        self.m = m
        self.H = H

        self.Q = nn.Parameter(torch.empty(H, m, n))
        self.K = nn.Parameter(torch.empty(H, m, n))
        self.V = nn.Parameter(torch.empty(H, m, n))

        self.W = nn.Parameter(torch.empty(n, m))
        
        # First layer normalization object.
        # Layernorm will average over the n dimensions of each element in the sequence.
        self.layer_norm1 = nn.LayerNorm(n)
        
        self.nonlinearity = nn.ReLU()
        
        # Second layer normalization object.
        self.layer_norm2 = nn.LayerNorm(n)
        
        self.initialize_parameters()

    def initialize_parameters(self):
        """
        Initialize the values of the learnable parameter matrices.
        Kaiming uniform is just a type of random initialization, you don't need to 
        worry about it. It is a good default initialization for linear layers.
        """
        nn.init.kaiming_uniform_(self.Q, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.K, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.V, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.W, a=math.sqrt(5))

    def forward(self, X):
        """
        Forward pass of the multihead attention layer with layer normalization.

        Args:
            X (torch.Tensor): The input embeddings.
        Returns:
            X_l (torch.Tensor): The output of the multihead attention layer.
        """
        B, n, T = X.shape  # X: (B, n, T)

        # First layer normalization.
        # An annoying Pytorch detail: layer norm function expects the normalization to be over the last dimension.
        # Therefore, we need to transpose the last two dimensions of the input to shape (B, T, n) each time we normalize, then transpose back.
        # (X.transpose(-2,-1) means that we are transposing over the last two dimensions)
        X = self.layer_norm1(X.transpose(-2,-1)).transpose(-2,-1)

        # Expand X to include the head dimension
        X_expanded = X.unsqueeze(1)  # (B, 1, n, T)

        # Compute QX, KX, VX for each head
        QX = torch.matmul(self.Q.unsqueeze(0), X_expanded)  # (B, H, m, T)
        KX = torch.matmul(self.K.unsqueeze(0), X_expanded)  # (B, H, m, T)
        VX = torch.matmul(self.V.unsqueeze(0), X_expanded)  # (B, H, m, T)
        
        QX_t = QX.transpose(-2, -1)  # (B, H, T, m)

        # Compute attention scores B per head
        B = torch.matmul(QX_t, KX)  # (B, H, T, T)
        A = F.softmax(B, dim=-1)
    
        A_t = A.transpose(-2,-1)
        VXA_t = torch.matmul(VX, A_t) # (B, H, m, T)
        Y = torch.matmul(self.W, VXA_t) # (B, H, T, n)

        # Second layer normalization. Transpose over the last two dimensions
        Y = self.layer_norm2(Y.transpose(-2,-1)).transpose(-2,-1)
        
        X_l = X + self.nonlinearity(Y.sum(dim=1))

        return X_l

# Testing the change
model = MultiHeadLayer(m=32, n=256, H=2).to(device)
X = torch.randn(1,256,5).to(device)
out = model(X)
print(f"out.shape: {out.shape}")

out.shape: torch.Size([1, 256, 5])


In [97]:

class LanguageTransformer(nn.Module):
    """
    Taken from Task 10 and added layer normalization. This is the final version of this class.
    """
    def __init__(self, m, n, L, H, c):
        super(LanguageTransformer, self).__init__()
        
        self.layers = nn.ModuleList([
            MultiHeadLayer(m, n, H) for _ in range(L)
        ])

        # PCA Word embeddings
        self.embedding_table = pca_embeddings
        
        # Positional encoding
        self.position_embedding = nn.Embedding(T, n)

        # Layer normalization
        self.layer_norm = nn.LayerNorm(n)

        # Adding readout layer
        self.readout = nn.Parameter(torch.empty(c, n).to(device))
        nn.init.kaiming_uniform_(self.readout, a=math.sqrt(5))
        
    def forward(self, E):
        """
        Args:
            E (torch.Tensor): The input word indices.
        Returns:
            Y_hat (torch.Tensor): The output of the transformer, passed through the readout layer.
        """
        B, T = E.shape

        # Word embeddings
        X = self.embedding_table[E].transpose(1,2) # (B, n, T)

        # To create positional encodings, we need to create a vector for each position in the sequence.
        P = self.position_embedding(torch.arange(T, device=device)).transpose(0,1) # (n, T)
        
        X_tilde = X + P
        
        # X_l has shape (B, n, T+1)
        X_l = X_tilde
        for layer in self.layers:
            X_l = layer(X_l)

        X_l = self.layer_norm(X_l.transpose(-2,-1)).transpose(-2,-1)

        # We implement the readout layer as a linear mapping on each word in the sequence.
        Y_hat = torch.matmul(self.readout, X_l) # (B, c, T)

        
        return Y_hat

# testing. 
E = torch.randint(0, pca_embeddings.shape[0], (1,5)).to(device).long()
print(f"E.shape: {E.shape}")
model = LanguageTransformer(m=32, n=256, L=6, H=8, c=c).to(device)
out = model(E)
print(f"out.shape: {out.shape}")

E.shape: torch.Size([1, 5])
out.shape: torch.Size([1, 14295, 5])


## Task 12: Future masking

In [95]:
# We need to modify both MultiHeadLayer and the LanguageTransformer class to include layer normalization.
class MultiHeadLayer(nn.Module):
    """
    A modified version of the MultiHeadLayer class with layer normalization.
    It will have two normalization layers, one after the multi-head attention and one after the nonlinearity.
    """
    def __init__(self, m, n, H):
        super(MultiHeadLayer, self).__init__()
        self.m = m
        self.H = H

        self.Q = nn.Parameter(torch.empty(H, m, n))
        self.K = nn.Parameter(torch.empty(H, m, n))
        self.V = nn.Parameter(torch.empty(H, m, n))

        self.W = nn.Parameter(torch.empty(n, m))
        
        # First layer normalization object.
        # Layernorm will average over the n dimensions of each element in the sequence.
        self.layer_norm1 = nn.LayerNorm(n)
        
        self.nonlinearity = nn.ReLU()
        
        # Second layer normalization object.
        self.layer_norm2 = nn.LayerNorm(n)
        
        self.initialize_parameters()

    def initialize_parameters(self):
        """
        Initialize the values of the learnable parameter matrices.
        Kaiming uniform is just a type of random initialization, you don't need to 
        worry about it. It is a good default initialization for linear layers.
        """
        nn.init.kaiming_uniform_(self.Q, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.K, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.V, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.W, a=math.sqrt(5))

    def forward(self, X):
        """
        Forward pass of the multihead attention layer with layer normalization.

        Args:
            X (torch.Tensor): The input embeddings.
        Returns:
            X_l (torch.Tensor): The output of the multihead attention layer.
        """
        B, n, T = X.shape  # X: (B, n, T)

        # First layer normalization.
        # An annoying Pytorch detail: layer norm function expects the normalization to be over the last dimension.
        # Therefore, we need to transpose the last two dimensions of the input to shape (B, T, n) each time we normalize, then transpose back.
        # (X.transpose(-2,-1) means that we are transposing over the last two dimensions)
        X = self.layer_norm1(X.transpose(-2,-1)).transpose(-2,-1)

        # Expand X to include the head dimension
        X_expanded = X.unsqueeze(1)  # (B, 1, n, T)

        # Compute QX, KX, VX for each head
        QX = torch.matmul(self.Q.unsqueeze(0), X_expanded)  # (B, H, m, T)
        KX = torch.matmul(self.K.unsqueeze(0), X_expanded)  # (B, H, m, T)
        VX = torch.matmul(self.V.unsqueeze(0), X_expanded)  # (B, H, m, T)
        
        QX_t = QX.transpose(-2, -1)  # (B, H, T, m)

        # Compute attention scores B per head
        B = torch.matmul(QX_t, KX)  # (B, H, T, T)

        # FUTURE MASKING: 
        # To mask attention, we create a matrix that indicates if an entry in B is a word in the future
        mask = torch.triu(torch.ones(T, T), diagonal=1).to(device)
        
        # If an entry is in the future, we set it to -inf, 
        # so that when we apply softmax, the probability of that word is 0, while 
        # the rest of the words sum to 1.
        B = B.masked_fill(mask == 1, float('-inf'))

        # Now when we apply softmax, only the words in the past are have nonzero probability.
        A = F.softmax(B, dim=-1)
    
        A_t = A.transpose(-2,-1)
        VXA_t = torch.matmul(VX, A_t) # (B, H, m, T)
        Y = torch.matmul(self.W, VXA_t) # (B, H, T, n)

        # Second layer normalization. Transpose over the last two dimensions
        Y = self.layer_norm2(Y.transpose(-2,-1)).transpose(-2,-1)
        
        X_l = X + self.nonlinearity(Y.sum(dim=1))

        return X_l

# Testing the change
model = MultiHeadLayer(m=32, n=256, H=2).to(device)
X = torch.randn(1,256,5).to(device)
out = model(X)
print(f"out.shape: {out.shape}")

out.shape: torch.Size([1, 256, 5])


In [79]:
# class MultiheadLayer(nn.Module):
#     def __init__(self, H, n=n):
#         super(MultiheadLayer, self).__init__()
        
#         assert n % H == 0, "n must be divisible by num_heads"
#         self.H = H
#         self.n = n
#         self.m = n // H

#         # Column wise
#         self.Q = nn.Parameter(torch.empty(H, self.m, n))
#         self.K = nn.Parameter(torch.empty(H, self.m, n))
#         self.V = nn.Parameter(torch.empty(H, self.m, n))

#         self.W = nn.Parameter(torch.empty(H, n, self.m))

#         self.norm1 = nn.LayerNorm(n)
#         self.norm2 = nn.LayerNorm(n)
        
#         nn.init.kaiming_uniform_(self.Q, a=math.sqrt(5))
#         nn.init.kaiming_uniform_(self.K, a=math.sqrt(5))
#         nn.init.kaiming_uniform_(self.V, a=math.sqrt(5))
#         nn.init.kaiming_uniform_(self.W, a=math.sqrt(5))

#     def forward(self, X0):
#         # Column wise
#         B, n, T = X0.shape
#         X = self.norm1(X0.transpose(1,2)).transpose(1,2) # (B, T, n)
#         X = X0
        
#         # Multi-head attention

#         X = X.unsqueeze(1) # (B, 1, T, n)
        
#         # Column wise
#         QX = torch.matmul(self.Q, X) # (B, H, T, m)
#         KX = torch.matmul(self.K, X) # (B, H, T, m)
#         VX = torch.matmul(self.V, X) # (B, H, T, m)
#         # 
#         B =  QX.transpose(-2,-1) @ KX * (self.m ** -0.5) # (B, H, T, T)

#         mask = torch.triu(torch.ones(T, T), diagonal=1).to(device)
#         B = B.masked_fill(mask == 1, float('-inf'))

#         A = F.softmax(B, dim=-1)
    
#         A_t = A.transpose(-2,-1)
#         VXA_t = torch.matmul(VX, A_t) # (B, H, m, T)
#         Y = torch.matmul(self.W, VXA_t) # (B, H, T, n)
#         X = torch.sum(Y, dim=1) # (B, T, n)

#         X = self.norm2(X.transpose(1,2)).transpose(1,2)
#         X2 = X0 + F.relu(X)
#         return X2
    
# # Test
# model = MultiheadLayer(H=1, n=32).to(device)
# #E = next(iter(train_loader))[0]
# #X = embeddings[E].transpose(2,1)
# # row wise
# # X = torch.randn(1,5, 32).to(device)
# # column wise
# X_tilde = torch.randn(1,5, 32).to(device).transpose(1,2)
# print(f"X.shape: {X_tilde.shape}")
# X_tilde = X_tilde.to(device)
# out = model(X_tilde)

# print(f"out.shape: {out.shape}")

X.shape: torch.Size([1, 32, 5])
out.shape: torch.Size([1, 32, 5])


## Final one

In [81]:

# class LLM(nn.Module):   
#     def __init__(self, L, H, n):
#         super(LLM, self).__init__()

#         self.position_embedding = nn.Embedding(T, n)
#         # this is the PCA embeddings 
#         self.token_embedding = pca_embeddings

#         self.decoder_layers = nn.Sequential(*[MultiheadLayer(H, n) for _ in range(L)])
        
#         self.norm = nn.LayerNorm(n)
        
#         # column wise
#         self.readout = nn.Parameter(torch.empty(c, n))
#         nn.init.kaiming_uniform_(self.readout, a=math.sqrt(5))

#     def forward(self, E):
#         B, T = E.shape

#         X = self.token_embedding[E].transpose(1,2) # (B, n, T)
#         P = self.position_embedding(torch.arange(T, device=device)).transpose(0,1) # (n, T)
        
#         X = X + P

#         X = self.decoder_layers(X) # (B, T, n)
#         X = self.norm(X.transpose(1,2)).transpose(1,2)

#         # column wise
#         Y = torch.matmul(self.readout, X) # (B, T, c)
#         # row wise
#         # Y = torch.matmul(X, self.readout_weight) 
        
#         return Y

# # Test
# model = LLM(L=2, H=2, n=n).to(device)
# initial_indices = test[132:132+T].unsqueeze(0)
# out = model(initial_indices)
# print(f"out.shape: {out.shape}")
# # generated_text = generate(model, initial, max_generate_tokens=100)
# # print("\n===INPUT===\n")
# # print(tokens_to_words(initial.reshape(-1).tolist()))
# # print("\n===GENERATED TEXT===\n")
# # print(generated_text)

IndexError: index 5974 is out of bounds for dimension 0 with size 1

In [ ]:
# def generate_text(model, input_tokens, max_generate_tokens=500):
#     """
#     Generate text from a model given an initial input token sequence.
#     Args:
#         model (nn.Module): The model to use for generation.
#         input_tokens (torch.Tensor): The initial input token sequence.
#         max_generate_tokens (int): The maximum number of tokens to generate.
#     Returns:
#         torch.Tensor: The generated token sequence.
#     """
#     with torch.no_grad():
#         context = input_tokens.clone()
#         generated_sequence = input_tokens.cpu().squeeze().tolist()  # Ensure it's a 1D list
#         for _ in range(max_generate_tokens):
#             logits = model(context)
            
#             last_token_logits = logits[:,-1,:]
#             probs = F.softmax(last_token_logits, dim=-1)
#             next_token = torch.multinomial(probs, num_samples=1)
            
#             # Slide context window

#             context = torch.cat([context[:, 1:], next_token], dim=1)  
#             generated_sequence.append(next_token.item())  # Add new token to generated sequence
#         generated_words = tokens_to_words(generated_sequence)
#         generation_string = "".join(generated_words)
#         return generation_string
# # Test generate
# model = LLM(L=2, H=4, n=n).to(device)
# initial_indices = test[132:132+T].unsqueeze(0)
# print(generate_text(model, initial_indices, max_generate_tokens=100))

mountains are for winds, 
 That shake not, though they blow perpetually. 
 
 BAPTISTA: 
 How now, my friend! why dost thou look so pale? 
 
 HORTENSIO: 
 For fear, I promise you, if I look pale. 
 
 BAPTISTA: 
 What, will my daughter prove a good musician whelp advanced hawks edges Adam's Aim'd Swills Seely Mock Swills taxes betwitched morning bestrid remembered watching commissions dumbly Mock surmounts masterless jest tougher jest lordship's Mock unlick'd affray edges lessons dumbly outside Aim'd morning wanders whelp whelp taxes Swills inforced wanders lordship's Mock piteous commissions banks sir's advanced GREEN Hannibal doting curious Clare misled separation hawks inforced Alla Hannibal misled masterless curious morning destroy remembered unlick'd wanders edges Aim'd Swills lessons lids destroy jest sir's disdained banks outside visit unlick'd disdained Travelling advanced Aim'd surmounts destroy wanders misled taxes jest notched Adam's piteous unlick'd affray lids separation sur

## Task 13: Final training
This final training loop has a slight difference from the ones used so far: 
We designed our Dataset to give us as targets all the next words for every word in $X$. This means we can evaluate cross entropy on the readouts of every token in the sequence,
comparing it to every element in $Y$. 

In [98]:

L = 6
H = 8
m=32
n=256
model = LanguageTransformer(m=m, n=n, L=L, H=H, c=c).to(device)
lr = 1e-4
num_epochs = 5

B = 32
train_loader = DataLoader(train_dataset, batch_size=B, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=B, shuffle=False)

num_parameters_str = str(sum(p.numel() for p in model.parameters())/1e6,) + 'M parameters'
print("Created new model with {}".format(num_parameters_str))
run = wandb.init(
    project="lab-9-llm",
    name="Varun's version",
    entity="ese-2000",
    config={
        "architecture": "Transformer",
        "dataset": "TinyShakespeare",
        "num_epochs": num_epochs,
        "learning_rate": lr,
        "num_blocks": L,
        "num_heads_per_block": H,
        "context_size": T,
        "model_summary": str(model),
        "num_parameters": num_parameters_str
    }
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


Created new model with 4.91136M parameters


wandb: Currently logged in as: javierporras (ese-2000). Use `wandb login --relogin` to force relogin


0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [99]:
from tqdm import trange, tqdm
opt = optim.AdamW(model.parameters(), lr=lr)
model.train()
num_parameters = str(sum(p.numel() for p in model.parameters())/1e6,) + 'M parameters'
print("Created new model with {} parameters".format(num_parameters))
train_loss_evolution = []
for epoch in trange(num_epochs):
    train_loss = 0
    for t_idx, (X_tilde, y) in enumerate(train_loader):
        logits = model(X_tilde)
        
        B, c, T = logits.shape
        
        # Reshape logits and y to be able to evaluate cross entropy on 
        # each token in the sequence.
        logits = logits.permute(0,2,1)
        logits = logits.reshape(B * T, c)
        y = y.view(B * T, -1).squeeze()
        cross_entropy_loss = F.cross_entropy(logits, y)
        opt.zero_grad()
        cross_entropy_loss.backward()
        opt.step()
        train_loss += cross_entropy_loss.item()
        wandb.log({"train_loss": cross_entropy_loss.item()})
    train_loss_evolution.append(train_loss/len(train_loader))
    clear_output()
    print(f"Epoch {epoch+1}, Loss {train_loss/len(train_loader)}")
    run.log({"epoch_train_loss": train_loss/len(train_loader)}) 
    wandb.config.update({"num_epochs": epoch+1})
    plt.plot(train_loss_evolution)
    plt.show()


Created new model with 4.91136M parameters parameters


  0%|          | 0/5 [00:00<?, ?it/s]

## Task 14: Final text generation

In [ ]:
initial_indices = test[132:132+T].unsqueeze(0)
generated_text = generate_text(model,initial_indices, max_generate_tokens=100)
print("\n===INPUT===\n")
print(tokens_to_words(initial_indices.reshape(-1).tolist()))
print("\n===GENERATED TEXT===\n")
print("".join(generated_text[:]))


===INPUT===

mountains are for winds, 
 That shake not, though they blow perpetually. 
 
 BAPTISTA: 
 How now, my friend! why dost thou look so pale? 
 
 HORTENSIO: 
 For fear, I promise you, if I look pale. 
 
 BAPTISTA: 
 What, will my daughter prove a good musician

===GENERATED TEXT===

mountains are for winds, 
 That shake not, though they blow perpetually. 
 
 BAPTISTA: 
 How now, my friend! why dost thou look so pale? 
 
 HORTENSIO: 
 For fear, I promise you, if I look pale. 
 
 BAPTISTA: 
 What, will my daughter prove a good musician? 
 
 NORTHUMBERLAND: 
 Believe to me, Buckingham; and see the truth of 
 Is all free to friend. Here 
 With age triumphs, my love still well 
 The heart of my brother's my word, 
 As he would put up. 
 
 PERDITA: 
 You will not need: 
 Yet, fair upon your highness. 
 
 First Citizen: 
 You are too bitterly: if the letter I have he: 
 That's his breast that have there have left the king: 
 And
